In [1]:
#!"C:\Users\Utilisateur\AppData\Local\Programs\Python\Python311\python.exe" -m pip install nbformat

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import when, avg, mean, round
import plotly.express as px
import numpy as np

## - Initialisation de la session Spark -

On vérifie également que le .csv est bien récupéré

In [3]:
spark = SparkSession.builder.getOrCreate()

In [4]:
df = spark.read.csv(r"Data/Sept_20_analysis.csv", header=True, inferSchema=True) # inferSchema est pour cast automatiquement en int
df.show()


+----+--------+---------------+----------+---+--------------------+--------------------+------+--------------------+------------+-----------+-------------------+--------+---------------+------------------+------------------+-------------+---------+---------+-----------+--------------+--------------+--------------+--------------+------------------+------------------+--------------------+--------------------+--------------+--------------+-----------------+-----------------+-----------------+----------------+----------------+----------------+--------------------+--------------------+----------+-------------+
|GAME|BlackElo|BlackRatingDiff|      Date|ECO|               Event|             Opening|Result|                Site| Termination|TimeControl|            UTCTime|WhiteElo|WhiteRatingDiff|Black_elo_category|White_elo_category|starting_time|increment|Game_type|Total_moves|Black_blunders|White_blunders|Black_mistakes|White_mistakes|Black_inaccuracies|White_inaccuracies|Black_inferior_move

## - Q1 - 

### What is the rate of blunders, errors and inaccuracies per move, per level category (*) and on Blitz type games (Blitz type is by far the most played on these online sites). A game has two players, whose ELOs are most likely different. You will be able to classify a game into a category, either by considering the average ELO of both players, or by considering only the games where both players are in the same category

Hypothèse : On peut largement imaginer que plus la partie a un niveau élevée, plus la proportion de mauvais coups va diminuer. Et cela même si on garde les parties désiquilibrées (on peut imaginer qu'un joueur bon peut toujours commetre des imprécisions contre un joueur excellent).

Pour ça, on a d'abord choisi d'ajouter une colonne calculant la moyenne des ELOs des 2 joueurs. Nous pouvons alors classer la partie dans une des classes données en annexe (on a ajouté une classe Beginner qui correspondait à de nombreux cas et élargie la classe GMI) :  
[0-1199] beginner  
[1200-1499] occasional player  
[1500-1799] good club player  
[1800-1999] very good club player  
[2000-2399] national and international level (IM)  
[>2400] GMI, World Champions



In [5]:
df = df.withColumn('MeanElo', (df.BlackElo + df.WhiteElo)/2)
niveauPartie = {"Beginner" : 0,
               "Occasional" : 1200,
               "Good": 1500,
               "Very Good" : 1800,
               "National" : 2000,
               "GMI": 2400}

df = df.withColumn(
    "niveauPartie",
    when(df.MeanElo >= 2400, "GMI")
    .when(df.MeanElo  >= 2000, "National")
    .when(df.MeanElo  >= 1800, "Very Good")
    .when(df.MeanElo  >= 1500, "Good")
    .when(df.MeanElo  >= 1200, "Occasional")
    .otherwise("Beginner") # pour les ELOs < 1200
)

# order_map nous servira lorsqu'on ordonnera les classes pour que ca respecte le classement plutôt que l'ordre alphabétique
order_map = {
    'beginner': 1,
    'ocasional': 2,
    'good': 3,
    'very good': 4,
    'national': 5,
    'GMI': 6
}

In [6]:
df.describe(["BlackElo", "WhiteElo", "MeanElo", "niveauPartie"]).show()
df.show(10)

+-------+------------------+------------------+------------------+------------+
|summary|          BlackElo|          WhiteElo|           MeanElo|niveauPartie|
+-------+------------------+------------------+------------------+------------+
|  count|           3739909|           3739909|           3739909|     3739909|
|   mean|1602.3001321689912|1601.5121897350978|1601.9061609520445|        NULL|
| stddev| 343.5793362601174|343.58978967446455| 335.7342182609751|        NULL|
|    min|               600|               600|             600.0|    Beginner|
|    max|              3957|              3958|            3492.5|   Very Good|
+-------+------------------+------------------+------------------+------------+

+----+--------+---------------+----------+---+--------------------+--------------------+------+--------------------+------------+-----------+-------------------+--------+---------------+------------------+------------------+-------------+---------+---------+-----------+---------

On peut désormais ajouter la proportion de blunders, d'erreurs et d'imprécisions par coup. Cela consiste à additionner les blunders noirs et blancs (de même pour les erreurs et les imprécisions), puis à diviser le total par le nombre total de coups. Cela nous permet ainsi d'obtenir la proportion de blunders, d'erreurs et d'imprécisions.

In [7]:
df = df.withColumn('RateBlunders', (df.Black_blunders + df.White_blunders)/df.Total_moves)
df = df.withColumn('RateMistakes', (df.Black_mistakes + df.White_mistakes)/df.Total_moves)
df = df.withColumn('RateInnacuracies', (df.Black_inaccuracies + df.White_inaccuracies)/df.Total_moves)

Maintenant qu'on a le DataFrame amélioré des classes représentant le niveaux des parties, on peut répondre à la problématique

In [8]:
df_blitz = df.filter(df.Game_type == "Blitz")

In [9]:
Q1 = df_blitz.groupby('niveauPartie').agg(
    round(mean('RateBlunders'),3).alias("Rate Blunder"),
    round(mean('RateMistakes'),3).alias("Rate Mistake"),
    round(mean('RateInnacuracies'),3).alias("Rate Inaccuracy")
)

Q1 = Q1.withColumn(
    'niveau_order', 
    when(df.niveauPartie == 'Beginner', 1)
    .when(df.niveauPartie == 'Occasional', 2)
    .when(df.niveauPartie == 'Good', 3)
    .when(df.niveauPartie == 'Very Good', 4)
    .when(df.niveauPartie == 'National', 5)
    .when(df.niveauPartie == 'GMI', 6)
)

# Trier selon l'ordre défini
Q1 = Q1.orderBy('niveau_order')

Q1.show()


+------------+------------+------------+---------------+------------+
|niveauPartie|Rate Blunder|Rate Mistake|Rate Inaccuracy|niveau_order|
+------------+------------+------------+---------------+------------+
|    Beginner|       0.097|        0.11|          0.094|           1|
|  Occasional|       0.076|       0.106|          0.094|           2|
|        Good|       0.061|       0.099|          0.094|           3|
|   Very Good|        0.05|       0.092|          0.091|           4|
|    National|       0.041|       0.082|          0.085|           5|
|         GMI|       0.032|       0.067|          0.075|           6|
+------------+------------+------------+---------------+------------+



In [10]:
result_pandas = Q1.toPandas()

# Créer le graphique avec Plotly
fig = px.line(result_pandas, x='niveauPartie', 
              y=['Rate Blunder', 'Rate Mistake', 'Rate Inaccuracy'],
              labels={'niveauPartie': 'Niveau de Partie', 
                      'value': 'Rate', 
                      'variable': 'Type d\'Erreur'},
              title='Rates of Blunders, Mistakes, and Inaccuracies by NiveauPartie')

# Afficher le graphique
fig.show()